In [16]:
#Step 1: Getting historical Data from yfinance
import numpy as np
import pandas as pd
import yfinance as yf

assets = ['ES=F', 'CL=F', 'GC=F', 'SI=F', 'AAPL', 'MSFT', 'GOOGL', 'AMZN', 
          'JPM', 'V', 'PG', 'XOM', 'GLD', 'SLV', 'USO', 'SPY', 'TLT', 'QQQ', 'DIA', 
          'NFLX', 'TSLA', 'BA', 'KO', 'DIS', 'PLUG', 'GS', 'IBM', 'INTC', 'ORCL', 'NVDA']

start_date = '2000-01-01'
end_date = '2021-12-31'
# Fetch the data for all assets
data = pd.DataFrame()
for i,  asset in enumerate(assets):
    try:
        df = yf.download(asset, start=start_date, end=end_date)  # download data for each asset
        if not df.empty:  
            df = df.resample('B').mean().fillna(method='ffill')  
            data[asset] = df['Close'].pct_change()  # Store close prices in the data dataframe
            
    except Exception as e:
        print(f"Failed to download {asset}: {str(e)}")

data = data.dropna()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [17]:
#Step 2: calculating the covariance matrix
cov_matrix = np.cov(data[assets].values, rowvar=False)

#Step 3: calculating the shrunken covariance matrix
#diagonal matrix
    # use np.diag twice: first, to extract the diagonal (variances) from the covariance matrix 
    # second: to construct a new matrix with these variances on the diagonal and zeros elsewhere. 
D = np.diag(np.diag(cov_matrix)) 

#shrinkage intensity
    #a hyperparameter that determines the degree of shrinkage
    #0<gamma<1
gamma = 0.5

#shrinkage covariance matrix
cov_matrix_shrunk = gamma * D + (1 - gamma) * cov_matrix

#Step 4: calculate eigenvalues
    #give information about the amount of variance in the data. Larger eigenvalues correspond to larger variances. 
    #In the context of finance, larger variance often means more risk.
eigvals_original = np.linalg.eigvals(cov_matrix)
eigvals_shrunk = np.linalg.eigvals(cov_matrix_shrunk)

print(f'Eigenvalues of original covariance matrix: {eigvals_original}')
print(f'Eigenvalues of shrunk covariance matrix: {eigvals_shrunk}')

Eigenvalues of original covariance matrix: [5.01868374e-03 3.73695622e-03 2.43605759e-03 9.69309468e-04
 8.33112139e-04 7.34091814e-04 4.91233735e-04 3.16171047e-04
 2.99687736e-04 2.31940076e-04 1.96504178e-04 1.62102242e-04
 1.55660812e-04 1.34210203e-04 1.11274178e-06 5.26271994e-06
 3.50995921e-06 4.40154114e-06 1.91705377e-05 1.14228712e-04
 1.08214827e-04 1.04150587e-04 9.25514207e-05 8.33319193e-05
 7.67586046e-05 6.56644820e-05 4.95279632e-05 4.11392111e-05
 4.32028882e-05 4.52982458e-05]
Eigenvalues of shrunk covariance matrix: [4.44789946e-03 3.53651932e-03 1.76394617e-03 9.86464591e-04
 8.44993316e-04 5.69055095e-04 5.30953544e-04 4.15155755e-04
 3.71196075e-04 3.11005166e-04 2.59374408e-04 5.31249489e-05
 5.55457876e-05 5.81950863e-05 6.62683675e-05 7.06605689e-05
 8.40336670e-05 8.02111297e-05 9.44477576e-05 1.13226949e-04
 2.34729244e-04 2.15096353e-04 1.48341270e-04 1.96383297e-04
 1.65244361e-04 1.88771028e-04 1.87501017e-04 1.70585282e-04
 1.78595079e-04 1.75723271e-04

In [18]:
#Step 5: calculating variances
#From the results:
    #the eigenvalues are not consistently lower or higher, so to get more concrete results, we should calculate the respective variances
variance_original = np.var(eigvals_original)
variance_shrunk = np.var(eigvals_shrunk)

print(f'Variance of eigenvalues of original covariance matrix: {variance_original}')
print(f'Variance of eigenvalues of shrunk covariance matrix: {variance_shrunk}')

Variance of eigenvalues of original covariance matrix: 1.2921670141405038e-06
Variance of eigenvalues of shrunk covariance matrix: 9.80645231655608e-07


In [19]:
#From the results, can conclude that the shrunk covariance matrix has a lower variance
#This indicates lower risk and hence suggests a more stable protfolio, making it more optimal for portfolio management